In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
df = pd.read_csv('Dataset\dogecoin_10_years.csv')

# Drop rows with missing values
df.dropna(inplace=True)

# Features and target
X = df[['market_cap', 'volume']]  # You can add more features if available
y = df['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Squared Error: 0.0000
R² Score: 0.9995


In [9]:
print(df[['price', 'market_cap', 'volume']].corr())


               price  market_cap    volume
price       1.000000    0.999743  0.629009
market_cap  0.999743    1.000000  0.623960
volume      0.629009    0.623960  1.000000


From the correlation matrix, we can see that market cap and price is correlated. Now there are two options:
- Building a time aware model: Model depend on the past data instead of the derived current values.
- Remove market cap from the features
I will focus more one first option as it could be more realistic

# Building a time-aware model

## Using market cap lag 1

In [ ]:
# Shifting features one day back to create lag features and time series analysis
df['market_cap_lag1'] = df['market_cap'].shift(1)
df['volume_lag1'] = df['volume'].shift(1)
df.dropna(inplace=True)

X = df[['market_cap_lag1', 'volume_lag1']]
y = df['price']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Squared Error: 0.0001
R² Score: 0.9855


## Using market cap lag 1, price lag 1

In [23]:
# Shifting features one day back to create lag features and time series analysis
df['price_lag1'] = df['price'].shift(1)
df['market_cap_lag1'] = df['market_cap'].shift(1)
df['volume_lag1'] = df['volume'].shift(1)
df.dropna(inplace=True)

X = df[['price_lag1', 'market_cap_lag1', 'volume_lag1']]
y = df['price']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Squared Error: 0.0001
R² Score: 0.9871


We can see that $MSE$ and $R^2$ error showing good result for both the cases in above. Another possible reason to get so accurate result is testing and splitting in random manner. Lets do it in chornological manner

## Splitting the data in chronological manner 

In [28]:
# Step 1: Sort by date (if not already sorted)
df['price_lag1'] = df['price'].shift(1)
df['market_cap_lag1'] = df['market_cap'].shift(1)
df['volume_lag1'] = df['volume'].shift(1)
df.dropna(inplace=True)
df = df.sort_values('date')

# Step 2: Define train size (for example: 80% for training)
train_size = int(len(df) * 0.7)

# Step 3: Split chronologically
X = df[['price_lag1', 'market_cap_lag1', 'volume_lag1']]
y = df['price']

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]

# Step 4: Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 5: Predict and evaluate
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Squared Error: 0.0002
R² Score: 0.9738


Next step to add more recent data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual Price', color='blue')
plt.plot(y_pred, label='Predicted Price', color='red', linestyle='--')
plt.title('Actual vs Predicted Dogecoin Prices')
plt.xlabel('Samples')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


,date,price,market_cap,volume,market_cap_lag1,volume_lag1
1,2013-12-17 00:00:00.000,0.000218,1619159.0,0.0,3488670.0,0.0
2,2013-12-18 00:00:00.000,0.000268,2191987.0,0.0,1619159.0,0.0
3,2013-12-19 00:00:00.000,0.000475,4299422.0,0.0,2191987.0,0.0
4,2013-12-20 00:00:00.000,0.000989,9866232.0,0.0,4299422.0,0.0
5,2013-12-21 00:00:00.000,0.000438,4686300.0,0.0,9866232.0,0.0
